# 🧠 Self-Reflecting Agent in Production using LangGraph

This notebook demonstrates a **production-style agentic AI pattern** using **LangGraph**.

✔ No LLM APIs
✔ No OpenAI / Anthropic calls
✔ Fully local ML model
✔ Real **self-reflection loop**

This is how self-reflection is **actually structured in production systems**.

## 🧩 Agent State (Shared Across LangGraph Nodes)

The state flows through the graph and is **mutated by each node**.

In [ ]:
from dataclasses import dataclass, field
from typing import List, Dict, Any

@dataclass
class AgentState:
    history: List[Dict[str, Any]] = field(default_factory=list)
    strategy: str = "default"
    done: bool = False

## 📦 Local ML Dependencies (Casual Model)

We intentionally use **simple local ML** to keep the focus on **agent behavior**, not model complexity.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## ⚙️ LangGraph Action Node: Train Model

In [ ]:
def train_model_node(state: AgentState) -> AgentState:
    X, y = make_classification(
        n_samples=200,
        n_features=5,
        class_sep=0.5 if state.strategy == "default" else 1.5,
        random_state=42
    )

    model = LogisticRegression()
    model.fit(X[:150], y[:150])

    preds = model.predict(X[150:])
    acc = accuracy_score(y[150:], preds)

    state.history.append({
        "node": "train_model",
        "strategy": state.strategy,
        "accuracy": acc
    })

    print(f"[Train] strategy={state.strategy} | accuracy={acc:.2f}")
    return state

## 🔁 LangGraph Reflection Node

This node **inspects prior actions** and decides whether to:

- Change strategy
- Continue execution
- Terminate the graph

In [ ]:
def reflection_node(state: AgentState) -> AgentState:
    last_action = state.history[-1]
    acc = last_action["accuracy"]

    print(f"[Reflect] evaluating accuracy={acc:.2f}")

    if acc < 0.75:
        print("[Reflect] accuracy low → switching strategy")
        state.strategy = "improved"
    else:
        print("[Reflect] accuracy acceptable → stopping graph")
        state.done = True

    state.history.append({
        "node": "reflection",
        "decision": state.strategy,
        "stop": state.done
    })

    return state

## 🧭 LangGraph Conditional Edge

This replaces `while` loops and controls **graph execution**.

In [ ]:
def should_continue(state: AgentState) -> str:
    return "end" if state.done else "train"

## 🕸️ Build the LangGraph

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("train", train_model_node)
graph.add_node("reflect", reflection_node)

graph.set_entry_point("train")

graph.add_edge("train", "reflect")

graph.add_conditional_edges(
    "reflect",
    should_continue,
    {
        "train": "train",
        "end": END
    }
)

agent_graph = graph.compile()

## ▶️ Run the Production-Style Agent

In [ ]:
final_state = agent_graph.invoke(AgentState())

print("\n--- FINAL AGENT MEMORY ---")
for h in final_state.history:
    print(h)

## 🧠 Why This Matches Production LangGraph Usage

- ✔ Explicit state object
- ✔ Modular nodes
- ✔ Conditional edges
- ✔ Reflection controls execution
- ✔ Drop-in LLM replacement later

This pattern scales directly to:
- LLM-as-judge reflection
- Tool-using agents
- Multi-agent graphs
- RAG pipelines